In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tqdm as tqdm
from PIL import Image
from pathlib import Path
from mtcnn import MTCNN

In [ ]:
import torchvision
import torch
from torchvision import datasets, models, transforms

In [ ]:
 trans = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [ ]:
model = torch.load("../model/model1.pt")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
detector = MTCNN()

In [ ]:
cap = cv2.VideoCapture(0)
frame_width = int(cap.get(3))    
frame_height = int(cap.get(4))
frame_area = frame_width * frame_height
margin = 20
while(cap.isOpened()):
    ret, frame1 = cap.read()
    if ret == True:
        frame = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
        box = detector.detect_faces(frame)
        if len(box) == 1:
            box = box[0]['box']
            face = np.copy(frame[box[1]-margin:box[1]+box[3]+margin , box[0]-margin:box[0]+box[2]+margin])
            if face.shape[0] != 0 and face.shape[1]!= 0 and face.shape[2] !=0:       
                im = Image.fromarray(face)
                inputs = trans(im)
                inputs = inputs.view(1,3,224,224)
                output = model(inputs.to(device))
                _, preds = torch.max(output, 1)
                if preds.cpu().numpy() == 1:
                    text = "OFF"
                else:
                    text = "ON"
                frame1 = cv2.rectangle(frame1, (box[0]-margin,box[1]-margin),(box[0]+box[2]+margin, box[1]+box[3]+margin), (255,255,255), 3)
                cv2.putText(frame1, text , (50,50),
                                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255) ,
                                        thickness=2, lineType=2)
            cv2.imshow('frame2',frame1)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
cv2.destroyAllWindows()
cap.release()

In [ ]:
cap.release()